## ufuncs: pandas e numpy
---

In [1]:
import pandas as pd
import numpy as np

qualquer ufunc visto em numpy vai funcionar aqui com series e dataframe de pandas, pois estes são contruídos baseados naquele.

In [2]:
rng = np.random.RandomState(42)
serie = pd.Series(rng.randint(0, 10, 4))
dataframe = pd.DataFrame(rng.randint(0, 10, (3, 4)), columns=['a', 'b', 'c', 'd'])

In [3]:
serie

0    6
1    3
2    7
3    4
dtype: int64

In [4]:
dataframe

,a,b,c,d
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


por exemplo

In [5]:
np.exp(serie)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [6]:
np.tan(dataframe)

,a,b,c,d
0,-0.291006,-0.452316,-2.185040,-0.291006
1,0.871448,1.157821,-0.142547,0.871448
2,0.871448,-2.185040,-3.380515,1.157821


In [7]:
dataframe**2 + np.pi

,a,b,c,d
0,39.141593,84.141593,7.141593,39.141593
1,52.141593,19.141593,12.141593,52.141593
2,52.141593,7.141593,28.141593,19.141593


#### alinhamento dos índices
---

quando trabalhando com operações binárias, pandas alinha os índices dos dois ou mais arrays durante a operação, evitando erros, por exemplo, quando falta informações.

por exemplo:

In [8]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')

as series acima dizem respeito à área e à população de alguns estados dos USA, respectivamente. só que, os dados não mostram os mesmos  estados, só alguns. Então, se for para calcular a densidade desses estados, o cálculo deve ocorrer apenas com aqueles estados que estão presentes em ambas as series. O pandas vai observar isso e corrigir os índices para fazer o cálculo de forma correta:

In [9]:
population/area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

observe que só há valores para os valores que intersectam as duas series:

In [10]:
area.index.intersection(population.index)

Index(['Texas', 'California'], dtype='object')

aqueles valores que não intersectam as series recebem `NaN`, que pode ser alterado, se desejar, com o parâmetro `fill_value`:

In [11]:
population.divide(area, fill_value=0)

Alaska         0.000000
California    90.413926
New York            inf
Texas         38.018740
dtype: float64

este alinhamento dos índices também ocorre com os dataframes:

In [12]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)), columns=list('AB'))
B = pd.DataFrame(rng.randint(0, 10, (3, 3)), columns=list('BAC'))

In [13]:
A + B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


ou, se desejar dar outro valor a `NaN`:

In [14]:
A.add(B, fill_value='inf')

,A,B,C
0,1.0,15.0,inf
1,13.0,6.0,inf
2,inf,inf,inf


as operações que pandas suporta, são:

python|array1.função(array2)
---|---
+|add
-|sub, subtract
*|mul, multiply
/|truediv, div, divide
//|floordiv
%|mod
**|pow

#### operações entre serie e dataframe
---

ocorre o mesmo processo do `broadcasting` do numpy:

In [15]:
A = pd.DataFrame(rng.randint(10, size=(2, 3)))
A

,0,1,2
0,3,8,2
1,4,2,6


In [16]:
A - A[0]

,0,1,2
0,0.0,4.0,NaN
1,1.0,-2.0,NaN


para fazer operações com as colunas, é necessário especificar através do parâmetro `axis=0`:

In [17]:
A.sub(A[0], axis=0)

,0,1,2
0,0,5,-1
1,0,-2,2


aqui, também, ocorre o alinhamento de índices entre a serie e o dataframe.

#### operação de strings vetorizadas
---

o numpy não consegue fazar as ufuncs funconar com strings:

In [18]:
data = np.array(['caçapa', 'broca', 'succinto', 'luxor', 'odessa', 'luanda'])
data.capitalize()

AttributeError: 'numpy.ndarray' object has no attribute 'capitalize'

nestes casos, é necessário fazer um loop, que dependendo do tamanho do dataframe, pode ser demorado ou impossível de calcular:

In [19]:
data = np.array(['caçapa', 'broca', 'succinto', 'luxor', 'odessa', 'luanda'])
[s.capitalize() for s in data]

['Caçapa', 'Broca', 'Succinto', 'Luxor', 'Odessa', 'Luanda']

nem esta opção mais manual funciona 100% das vezes, já que se houver um valor faltando, uma exceção ocorre:

In [20]:
data = np.array(['caçapa', 'broca', 'succinto', None, 'luxor', 'odessa', 'luanda'])
[s.capitalize() for s in data]

AttributeError: 'NoneType' object has no attribute 'capitalize'

o pandas, por sua vez, consegue generalizar as ufuncs para as strings:

In [21]:
name = pd.Series(['caçapa', 'broca', 'succinto', None, 'luxor', 'odessa', 'luanda'])
name

0      caçapa
1       broca
2    succinto
3        None
4       luxor
5      odessa
6      luanda
dtype: object

assim,

In [22]:
name.str.capitalize()

0      Caçapa
1       Broca
2    Succinto
3        None
4       Luxor
5      Odessa
6      Luanda
dtype: object

observe que, mesmo com a presença de um valor nulo, usando pandas funciona.

É, no entanto, sempre necessário usar o atributo `str` para que funciona.

Os métodos de pandas são, em sua maioria, iguais ao métodos nativos do python para lidar com strings:
método()|método()|método()|método()
---|---|---|---
len|lower|translate|islower
ljust|upper|startswith|isupper
rjust|find|endswith|isnumeric
center|rfind|isalnum|isdecimal
zfill|index|isalpha|split
strip|rindex|isdigit|rsplit
rstrip|capitalize|isspace|partition
lstrip|swapcase|istitle|rpartition

outros métodos de pandas que não são nativos do python são
método()|descrição
---|---
get|endereça cada elemento
slice|divide cada elemento
slice_replace|substitui cada divisão pelo valor passado
cat|concatena strings
repeat|repete valores
normalize|retorns o código unicode da string
pad|adiciona whitespace nos lados esquerdos, direitos ou nos dois da string
wrap|divide longas strings em um tamanho menor que o valor passado
join|junta strings de cada elemento da series com os separadores passados
get_dummies|extrai variáveis *dummies* como uma dataframe

assim, por exemplo, podemos usar `.slice()` para que só mostre algumas letras de cada nome da series exemplo:

In [23]:
name.str.slice(0, 3)

0     caç
1     bro
2     suc
3    None
4     lux
5     ode
6     lua
dtype: object

neste exemplo, usar apenas o indexamento nativo do python funciona:

In [24]:
name.str[0:3]

0     caç
1     bro
2     suc
3    None
4     lux
5     ode
6     lua
dtype: object

e note que é diferente do slice direto na series:

In [25]:
name[0:3]

0      caçapa
1       broca
2    succinto
dtype: object

`.get()` retorna apenas uma letra de cada valor:

In [26]:
name.str.get(1)

0       a
1       r
2       u
3    None
4       u
5       d
6       u
dtype: object

inclusive, pode ser usado com valores abstratos:

In [27]:
name.str.get(-1)

0       a
1       a
2       o
3    None
4       r
5       a
6       a
dtype: object

ou, por exemplo, pegar apenas o último nome:

In [28]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam','Eric Idle', 'Terry Jones', 'Michael Palin'])
monte

0    Graham Chapman
1       John Cleese
2     Terry Gilliam
3         Eric Idle
4       Terry Jones
5     Michael Palin
dtype: object

In [29]:
monte.str.split().str.get(-1)

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

o método `.get_dummies()` serve para separar informações, por exemplo, codificadas:

In [30]:
full_monty = pd.DataFrame({'nome': monte, 'info':['B|C|D', 'B|D', 'A|C', 'B|D', 'B|C', 'B|C|D']})
full_monty

,nome,info
0,Graham Chapman,B|C|D
1,John Cleese,B|D
2,Terry Gilliam,A|C
3,Eric Idle,B|D
4,Terry Jones,B|C
5,Michael Palin,B|C|D


assim, 

In [31]:
full_monty['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


a manuipulação de string no pandas é importante, pois dados da vida real são mais bagunçados e alguns podem precisar de uma organização para que as informações sejam encontradas, e isto pode ser feito com estes métodos de string.

#### panda de alta perfomance
---

como para fazer operações entre datasets, pandas ocupa temporariamente memória, as operações podem se tornar lentas quanto maiores forem os datasets. Por isso, pandas tem os métodos `.eval()` e `.query()`, que ajudam na performance das operações. Seguindo o exemplo, onde são criados quatro dataframes consideravelmente grandes:

In [32]:
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

agora, observe o tempo que leva para calcular a soma desses quatro datraframes:

In [33]:
%timeit df1 + df2 + df3 + df4

104 ms ± 3.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


compare, então, com a mesma operação só que feita através do método `.eval()`:

In [34]:
%timeit pd.eval('df1 + df2 + df3 + df4')

103 ms ± 371 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


o método `.eval()` recebe, como uma string, as operações que o usuário deseja fazer.

Este método suporta
1. todas as operações aritméticas. Por exemplo, `pd.eval('-df1 * df2 / (df3 + df4) - df5')`;
2. todas as operações de comparção. Por exemplo, `pd.eval('df1 < df2 <= df3 != df4')`;
3. todas as operações binárias. Por exemplo, `pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')`, inclusive pode substituir `&` por `and` e `|` por `or`;
4. todas as operações de indexing e slicing. Por exemplo, `pd.eval('df2.T[0] + df3.iloc[1]')`;
5. dentre outras operações, como chamada de funções, condicionais, loops, etc.

para trabalhar com colunas usando `.eval()`, em dataframes, há duas formas possíveis:

In [35]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df

,A,B,C
0,0.615875,0.525167,0.047354
1,0.330858,0.412879,0.441564
2,0.689047,0.559068,0.230350
3,0.290486,0.695479,0.852587
4,0.424280,0.534344,0.245216
...,...,...,...
995,0.746470,0.545752,0.614236
996,0.223362,0.748549,0.550715
997,0.047455,0.450104,0.468734
998,0.895440,0.894235,0.209260


em uma forma, `eval()` é chamado através do próprio pandas, e cada coluna é chamada como um atributo do dataframe:

In [36]:
pd.eval("(df.A + df.B) / (df.C - 1)")

0     -1.197761
1     -1.331822
2     -1.621667
3     -6.688481
4     -1.270064
         ...   
995   -3.349773
996   -2.163240
997   -0.936554
998   -2.263292
999   -3.781258
Length: 1000, dtype: float64

na outra forma, `.eval()` é chamado através do dataframe em que se está trabalhando, e as colunas são chamadas como variáveis:

In [37]:
df.eval('(A + B) / (C-1)')

0     -1.197761
1     -1.331822
2     -1.621667
3     -6.688481
4     -1.270064
         ...   
995   -3.349773
996   -2.163240
997   -0.936554
998   -2.263292
999   -3.781258
Length: 1000, dtype: float64

pode-se, inclusve, adicionar novas informações ao dataset usando `.eval()`, como, por exemplo, adicionar uma coluna `D` no dataframe:

In [38]:
df.eval('D = (A + B) / C', inplace=True)
df

,A,B,C,D
0,0.615875,0.525167,0.047354,24.095868
1,0.330858,0.412879,0.441564,1.684325
2,0.689047,0.559068,0.230350,5.418335
3,0.290486,0.695479,0.852587,1.156439
4,0.424280,0.534344,0.245216,3.909296
...,...,...,...,...
995,0.746470,0.545752,0.614236,2.103789
996,0.223362,0.748549,0.550715,1.764818
997,0.047455,0.450104,0.468734,1.061497
998,0.895440,0.894235,0.209260,8.552382


ou, ainda, modificar informações já no dataframes:

In [39]:
df.eval('D = (A - B) / C', inplace=True)
df

,A,B,C,D
0,0.615875,0.525167,0.047354,1.915527
1,0.330858,0.412879,0.441564,-0.185752
2,0.689047,0.559068,0.230350,0.564268
3,0.290486,0.695479,0.852587,-0.475016
4,0.424280,0.534344,0.245216,-0.448844
...,...,...,...,...
995,0.746470,0.545752,0.614236,0.326776
996,0.223362,0.748549,0.550715,-0.953646
997,0.047455,0.450104,0.468734,-0.859015
998,0.895440,0.894235,0.209260,0.005759


pode, até mesmo, ter acesso às variáveis do código python, usando `@`:

In [40]:
column_mean = df.sum()
df.eval('A * @column_mean')
df

,A,B,C,D
0,0.615875,0.525167,0.047354,1.915527
1,0.330858,0.412879,0.441564,-0.185752
2,0.689047,0.559068,0.230350,0.564268
3,0.290486,0.695479,0.852587,-0.475016
4,0.424280,0.534344,0.245216,-0.448844
...,...,...,...,...
995,0.746470,0.545752,0.614236,0.326776
996,0.223362,0.748549,0.550715,-0.953646
997,0.047455,0.450104,0.468734,-0.859015
998,0.895440,0.894235,0.209260,0.005759


é, no entanto, importante observar que usar `@` para referenciar variáveis do python só serve quando `.eval()` é chamado através do dataframe. Se `.eval()` for chamado pelo pandas, não funcionará.

ainda assim, `.eval()` não funciona para todos os casos, como, por exemplo, quando deseja que o dataset retorne algum valor, como num slicing ou indexing, para isso, deve ser usado o `.eval()` chamado pelo pandas ou usado o método `.query()`, que sempre será chamado pelo dataset:

In [41]:
df.query('A < 0.5 and B < 0.5') 

,A,B,C,D
1,0.330858,0.412879,0.441564,-0.185752
8,0.448611,0.415924,0.481001,0.067958
10,0.112910,0.394884,0.950129,-0.296774
11,0.191011,0.118751,0.130223,0.554895
14,0.075723,0.260648,0.956146,-0.193407
...,...,...,...,...
964,0.478935,0.196736,0.913372,0.308964
967,0.498382,0.465993,0.664128,0.048768
980,0.150918,0.382386,0.305427,-0.757852
982,0.207822,0.356162,0.653230,-0.227087


esta linha faz o mesmo que:

In [42]:
df[(df['A'] < 0.5) & (df['B'] < 0.5)]

,A,B,C,D
1,0.330858,0.412879,0.441564,-0.185752
8,0.448611,0.415924,0.481001,0.067958
10,0.112910,0.394884,0.950129,-0.296774
11,0.191011,0.118751,0.130223,0.554895
14,0.075723,0.260648,0.956146,-0.193407
...,...,...,...,...
964,0.478935,0.196736,0.913372,0.308964
967,0.498382,0.465993,0.664128,0.048768
980,0.150918,0.382386,0.305427,-0.757852
982,0.207822,0.356162,0.653230,-0.227087


`.query()` também recebe `@` para chamar variáveis locais do python.